In [1]:
%load_ext autoreload
%autoreload 2

In [21]:
import numpy as np
import random 

In [36]:
from rmab.simulator import RMABSimulator, random_valid_transition, random_valid_transition_round_down, synthetic_transition_small_window
from rmab.uc_whittle import UCWhittle
from rmab.ucw_value import UCWhittle_value
from rmab.baselines import optimal_policy, random_policy, WIQL


In [27]:
n_arms      = 8
budget      = 3
n_states    = 2
n_actions   = 2

# solution/evaluation setup
discount    = 0.9
alpha       = 3 #7 - too pessimistic #0.1 - too optimistic

# experiment setup
seed        = 42
VERBOSE     = False
LOCAL       = False 
prefix      = ''
n_episodes  = 30
episode_len = 20
n_epochs    = 10
data        = 'synthetic'

In [28]:
np.random.seed(seed)
random.seed(seed)

In [29]:
all_population_size = 100 # number of random arms to generate
all_transitions = random_valid_transition(all_population_size, n_states, n_actions)

In [30]:
# For each member of the population, what's the probability they go from s -> a -> s'
# Normalized, so for each (population,s,a), the probabilities sum to 1
np.sum(all_transitions[0][0]), all_transitions.shape

(2.0, (100, 2, 2, 2))

In [31]:
all_features = np.arange(all_population_size)

In [32]:
simulator = RMABSimulator(all_population_size, all_features, all_transitions,
            n_arms, episode_len, n_epochs, n_episodes, budget, number_states=n_states)

acting should always be good! 0.977 < 0.992
acting should always be good! 0.846 < 0.850
acting should always be good! 0.997 < 0.999
acting should always be good! 0.325 < 0.630
acting should always be good! 0.914 < 0.963
acting should always be good! 0.993 < 0.996
acting should always be good! 0.986 < 0.998
acting should always be good! 0.664 < 0.910
acting should always be good! 0.918 < 0.949
acting should always be good! 0.928 < 0.953
acting should always be good! 0.936 < 0.948
acting should always be good! 0.990 < 0.991
acting should always be good! 0.926 < 0.950
acting should always be good! 0.943 < 0.987
acting should always be good! 0.651 < 0.964
acting should always be good! 0.960 < 0.987
acting should always be good! 0.988 < 0.995
cohort [74 31 71 64 82 87 52 49]
cohort [12 92 22 83 29 14 27 82]
cohort [18 22  4  3 34 48 88 38]
cohort [74 39 57 60 37 22 51 77]
cohort [83 68 66 27 99 32 35 23]
cohort [31  1 98 11 36  4 84 58]
cohort [ 5 41 86 49 97 13 58 42]
cohort [22 71 78 83 3

In [33]:
simulator 

In [37]:
random_rewards = random_policy(simulator, n_episodes, n_epochs)

first state [1 0 1 0 0 0 1 0]
instance 0, ep 1, state [0 1 0 1 0 0 1 0]
instance 0, ep 2, state [0 0 0 1 0 1 0 1]
instance 0, ep 3, state [1 1 0 0 0 1 1 1]
instance 0, ep 4, state [0 1 1 0 0 1 1 1]
instance 0, ep 5, state [0 1 0 0 0 1 1 1]
instance 0, ep 6, state [0 1 0 1 0 0 0 0]
instance 0, ep 7, state [1 1 0 1 0 1 1 1]
instance 0, ep 8, state [0 1 1 0 0 1 1 1]
instance 0, ep 9, state [1 1 0 0 0 1 0 1]
instance 0, ep 10, state [1 1 0 1 0 1 0 0]
instance 0, ep 11, state [1 0 0 1 1 1 1 1]
instance 0, ep 12, state [0 0 0 1 1 0 0 1]
instance 0, ep 13, state [0 1 1 1 0 1 1 1]
instance 0, ep 14, state [1 1 1 0 1 1 0 1]
instance 0, ep 15, state [1 1 0 0 1 1 0 1]
instance 0, ep 16, state [1 1 1 0 0 0 1 0]
instance 0, ep 17, state [0 0 1 1 0 0 1 0]
instance 0, ep 18, state [0 0 0 0 0 0 1 0]
instance 0, ep 19, state [1 1 0 1 0 1 0 0]
instance 0, ep 20, state [0 1 1 1 0 1 0 1]
instance 0, ep 21, state [0 0 0 0 1 0 1 0]
instance 0, ep 22, state [1 0 0 1 0 0 1 1]
instance 0, ep 23, state [0 0 1 0

In [49]:
# For each epoch, for each episode x episode len

random_rewards.shape, np.mean(random_rewards)

((10, 601), 5.801830282861897)

In [50]:
optimal_reward = optimal_policy(simulator, n_episodes, n_epochs, discount)

first state [1 0 1 0 0 0 1 0]
   state [1 0 1 0 0 0 1 0] state_WI [  0.15   0.38   0.12   0.01   0.17   0.29   0.25 -10.  ] sorted [1 5 6]
   state [1 0 1 1 0 1 0 1] state_WI [  0.15   0.38   0.12   0.04   0.17   0.36   0.27 -10.  ] sorted [1 5 6]
   state [1 0 1 1 1 0 0 1] state_WI [  0.15   0.38   0.12   0.04   0.25   0.29   0.27 -10.  ] sorted [1 5 6]
   state [1 1 1 1 1 1 0 1] state_WI [  0.15   0.06   0.12   0.04   0.25   0.36   0.27 -10.  ] sorted [5 6 4]
   state [0 0 1 1 1 1 1 1] state_WI [  0.21   0.38   0.12   0.04   0.25   0.36   0.25 -10.  ] sorted [1 5 4]
   state [0 0 1 1 0 1 1 1] state_WI [  0.21   0.38   0.12   0.04   0.17   0.36   0.25 -10.  ] sorted [1 5 6]
   state [1 1 1 1 1 1 1 1] state_WI [  0.15   0.06   0.12   0.04   0.25   0.36   0.25 -10.  ] sorted [5 4 6]
   state [1 1 1 1 0 1 1 1] state_WI [  0.15   0.06   0.12   0.04   0.17   0.36   0.25 -10.  ] sorted [5 6 4]
   state [1 1 0 1 0 1 1 0] state_WI [  0.15   0.06   0.04   0.04   0.17   0.36   0.25 -10.  ] sort

In [53]:
np.mean(optimal_reward)

6.699334442595674